# A template notebook showing how to slew to a target. 

In [ ]:
import logging
import yaml

import numpy as np
from matplotlib import pyplot as plt
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5
import asyncio

# import palpy

from lsst.ts import salobj

from lsst.ts.standardscripts.auxtel.attcs import ATTCS

from lsst.ts.idl.enums import ATPtg

%matplotlib inline

In [ ]:
from astropy.utils import iers
iers.conf.auto_download = False

In [ ]:
d = salobj.Domain()

In [ ]:
atmcs = salobj.Remote(d, "ATMCS")
atptg = salobj.Remote(d, "ATPtg")
ataos = salobj.Remote(d, "ATAOS")
atpne = salobj.Remote(d, "ATPneumatics")
athex = salobj.Remote(d, "ATHexapod")
atdome = salobj.Remote(d, "ATDome", index=1)
atdomtraj = salobj.Remote(d, "ATDomeTrajectory")

In [ ]:
await asyncio.gather(atmcs.start_task, 
                     atptg.start_task,
                     ataos.start_task,
                     atpne.start_task,
                     athex.start_task,
                     atdome.start_task,
                     atdomtraj.start_task)

In [ ]:
await atdomtraj.cmd_enable.start()

In [ ]:
await atdome.cmd_moveAzimuth.set_start(azimuth=120.)

# ATMCS

If the ATMCS is not enable run the next cell.

In [ ]:
await asyncio.sleep(5.)
print("Done")
await salobj.set_summary_state(atmcs, salobj.State.ENABLED)

# ATPtg

If the Pointing Component is not enabled, run the next cell.

In [ ]:
await asyncio.sleep(5.)
print("Done")
await atptg.tel_timeAndDate.next(flush=True, timeout=5)
await salobj.set_summary_state(atptg, salobj.State.ENABLED)

# Going to the Target

The next cell sets the observatory location. This is needed to compute the Az/El of the target to set the camera rotation angle. We are trying to keep the angle close to zero. 

In [ ]:
location = EarthLocation.from_geodetic(lon=-70.747698*u.deg,
                                       lat=-30.244728*u.deg,
                                       height=2663.0*u.m)

This next cell defines a target.

In [ ]:
ra = Angle("20:25:38.85705", unit=u.hour)
dec = Angle("-56:44:06.3230", unit=u.deg)
target_name="Alf PAv"
radec = ICRS(ra, dec)

This next cell will slew to the target and set the camera rotation angle to zero. 

In [ ]:
# Figure out what is the rotPA that sets nasmith rotator close to zero.
time_data = await atptg.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_atptg = Time(time_data.tai, format="mjd", scale="tai")
print(curr_time_atptg)
coord_frame_altaz = AltAz(location=location, obstime=curr_time_atptg)
alt_az = radec.transform_to(coord_frame_altaz)

print("slew...")
# await atmcs.cmd_startTracking.start(timeout=10)
await atptg.cmd_raDecTarget.set_start(
    targetName=target_name,
    targetInstance=ATPtg.TargetInstances.CURRENT,
    frame=ATPtg.CoordFrame.ICRS,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=radec.ra.hour,
    declination=radec.dec.deg,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    rotPA=180.-alt_az.alt.deg,
    rotFrame=ATPtg.RotFrame.FIXED,
    rotMode=ATPtg.RotMode.FIELD,
    timeout=10
)

In case you need to stop tracking, use the next cell!

In [ ]:
await atptg.cmd_stopTracking.start(timeout=10)

Use the next cell in case you need to offset to center the target on the FoV.

This will set total offsets. So, if you say `el=0` and `az=-30` and then later you do `el=30` and `az=0.` , it will reset the offset in azimuth to zero and make an offset of 30arcs in elevation. 

In [ ]:
await atptg.cmd_offsetAzEl.set_start(el=0., 
                                     az=-100., 
                                         num=0)

If you want to make persistet offsets you can use the following method. 

In [ ]:
await atptg.cmd_offsetAzEl.set_start(el=0., 
                                     az=-100., 
                                     num=1)

If you want to add your offset to a pointing model file, do the following. 

In [ ]:
await atptg.cmd_pointNewFile.start()
await asyncio.sleep(1.)
await atptg.cmd_pointAddData.start()
await asyncio.sleep(1.)
await atptg.cmd_pointCloseFile.start()

In [ ]:
skypos = Angle(0., unit=u.deg)
target_name="Alf PAv"
radec = ICRS(Angle("20:25:38.85705", unit=u.hour), 
             Angle("-56:44:06.3230", unit=u.deg))

In [ ]:
await attcs.slew(ra="20:25:38.85705", 
                 dec="-56:44:06.3230", sky_pos=skypos.deg)